In [1]:
# %pip install ftfy regex tqdm
# %pip install colorama
%pip install scikit-learn

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
import jittor as jt
from PIL import Image
import jclip as clip
import os
from tqdm import tqdm
import argparse
from sklearn.linear_model import LogisticRegression
import numpy as np
# from colorama import Fore, Back, Style, init
# init()

jt.flags.use_cuda = 1
print("包导入成功")

[i 0614 19:20:29.175772 28 log.cc:351] Load log_sync: 1
[i 0614 19:20:29.237815 28 compiler.py:956] Jittor(1.3.8.5) src: /root/miniconda3/envs/pyt/lib/python3.10/site-packages/jittor
[i 0614 19:20:29.240216 28 compiler.py:957] g++ at /usr/bin/g++(9.4.0)
[i 0614 19:20:29.240772 28 compiler.py:958] cache_path: /root/.cache/jittor/jt1.3.8/g++9.4.0/py3.10.11/Linux-5.4.0-16xb5/IntelRXeonRPlax40/default
[i 0614 19:20:29.244014 28 __init__.py:411] Found /usr/local/cuda/bin/nvcc(11.7.99) at /usr/local/cuda/bin/nvcc.
[i 0614 19:20:29.246925 28 __init__.py:411] Found addr2line(2.34) at /usr/bin/addr2line.
[i 0614 19:20:29.332880 28 compiler.py:1011] cuda key:cu11.7.99_sm_70
[i 0614 19:20:29.682147 28 __init__.py:227] Total mem: 38.42GB, using 12 procs for compiling.
[i 0614 19:20:29.826940 28 jit_compiler.cc:28] Load cc_path: /usr/bin/g++
[i 0614 19:20:29.900677 28 init.cc:62] Found cuda archs: [70,]
[i 0614 19:20:32.712436 28 cuda_flags.cc:49] CUDA enabled.


包导入成功


In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--split', type=str, default='A')

# args = parser.parse_args()
args, unknown = parser.parse_known_args()
model, preprocess = clip.load("ViT-B-32.pkl")
classes = open('../data/classname.txt').read().splitlines()#这是一个包含所有类的列表

## 类别文本编码,变成一句话,然后别转为向量
### 提示词可以尝试优化
text_features的shape为[374,512,]

In [4]:
# encode这块后面可以强化下
new_classes = []
for c in classes:
    c = c.split(' ')[0]
    if c.startswith('Animal'):
        c = c[7:]
    if c.startswith('Thu-dog'):
        c = c[8:]
    if c.startswith('Caltech-101'):
        c = c[12:]
    if c.startswith('Food-101'):
        c = c[9:]
    c = 'a photo of ' + c
    new_classes.append(c)
    text = clip.tokenize(new_classes)
text_features = model.encode_text(text)
text_features /= text_features.norm(dim=-1, keepdim=True)
print("标签处理成功")

标签处理成功


## 图片加载并处理

In [5]:
# training data loading
imgs_dir = '../data'
train_labels = open('../data/train.txt').read().splitlines()
train_imgs = [l.split(' ')[0] for l in train_labels]#对应的图片path
train_labels = [jt.float32([int(l.split(' ')[1])]) for l in train_labels]#对应的种类序号

In [6]:
# 每个类挑四张图，根据train_labels中的label来挑选
#挑选每种的前四张,生成两个对应的列表,分别存储path和类别信息
cnt = {}
new_train_imgs = []
new_train_labels = []
for i in range(len(train_imgs)):
    label = int(train_labels[i].numpy())
    if label not in cnt:
        cnt[label] = 0
    if cnt[label] < 4:
        new_train_imgs.append(train_imgs[i])
        new_train_labels.append(train_labels[i])
        cnt[label] += 1

In [7]:
# calculate image features of training data
train_features = []
print('Training data processing:')
with jt.no_grad():
    for img in tqdm(new_train_imgs):
        img = os.path.join(imgs_dir, img)
        image = Image.open(img)
        image = preprocess(image).unsqueeze(0)
        image_features = model.encode_image(image)
        # print("能成功运行?","image_features的shape是",image_features.shape)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        train_features.append(image_features)

train_features = jt.cat(train_features).numpy()#(1496, 512)
train_labels = jt.cat(new_train_labels).numpy()#(1496,)

Training data processing:


100%|██████████| 1496/1496 [01:54<00:00, 13.11it/s]


In [13]:
# training
classifier = LogisticRegression(random_state=0,
                                C=8.960,
                                max_iter=1000,
                                verbose=1)
classifier.fit(train_features, train_labels)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       191862     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.92426D+00    |proj g|=  7.01077D-04


 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
*****     18     23      1     0     0   7.094D-05   2.982D+00
  F =   2.9816809769345811     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


LogisticRegression(C=8.96, max_iter=1000, random_state=0, verbose=1)

In [15]:
# testing dataset loading
split = 'TestSet' + args.split
imgs_dir = '../data/' + split
test_imgs = os.listdir(imgs_dir)

In [16]:
print('Testing data processing:')
test_features = []
with jt.no_grad():
    for img in tqdm(test_imgs):
        img_path = os.path.join(imgs_dir, img)
        image = Image.open(img_path)
        image = preprocess(image).unsqueeze(0)
        image_features = model.encode_image(image)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        test_features.append(image_features)

test_features = jt.cat(test_features).numpy()

Testing data processing:


100%|██████████| 3073/3073 [03:06<00:00, 16.51it/s]


In [17]:
# testing
with open('result_ft.txt', 'w') as save_file:
    i = 0
    predictions = classifier.predict_proba(test_features)
    for prediction in predictions.tolist():
        prediction = np.asarray(prediction)
        top5_idx = prediction.argsort()[-1:-6:-1]
        save_file.write(test_imgs[i] + ' ' +
                        ' '.join(str(idx) for idx in top5_idx) + '\n')
        i += 1